In [6]:
from collections import Counter
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
from datetime import date
import csv

In [7]:

class CrawledArticle():
    def __init__(self, topline, headline, content, link, release):
        self.topline = topline
        self.headline = headline
        self.content = content
        self.link = link
        self.release = release

def printArticles(listOfCrawledArticles):
     for article in listOfCrawledArticles:
        print("------")
        print(article.link)
        print(article.topline)
        print(article.headline)
        print(article.content)
        print("------")
        
def getArt(): #Get all Articles from Tagesschau.de
    url = "https://www.tagesschau.de/"
    
    time.sleep(0.2)
    t = requests.get(url)
    site = BeautifulSoup(t.text, "html.parser")
    
    print("Getting all Links...")
    listOfCrawledArticles = []
    for link in site.find_all('a'):
        #Sort out all useless articles
        if link.get('href')[0] == "#" or link.get('href')[0] != "h" or link.get('href')[-4:] != "html" or link.parent is None:
            continue
        elif link.parent.select(".teaser__topline") == [] or link.select(".teaser__headline")[0].text == "tagesthemen" or link.select_one(".teaser__topline").text == "ARD-Programm":
            continue
        elif "tagesschau" not in link.get('href'):
            continue
        else:
            headline = link.select(".teaser__headline")[0].text
            linkHref = link.get('href')
            topline = link.parent.select(".teaser__topline")[0].text
            release = #TODO
            crawled = CrawledArticle(topline, headline, "", linkHref, #TODO Release)
            listOfCrawledArticles.append(crawled)
            
    print("Got " + str(len(listOfCrawledArticles)) + " articles...")
    getContent(listOfCrawledArticles)
    return listOfCrawledArticles

def getContent(listOfArticles):
    for article in listOfArticles:
        print("Getting content of article: " + article.headline)
        contentOfArticle = ""
        URL = article.link
        time.sleep(0.2)
        t = requests.get(URL)
        site = BeautifulSoup(t.text, "html.parser")
        
        for paragraph in site.find_all('p'):
            
            if bool(paragraph.attrs) is False:
                continue
            elif paragraph.attrs["class"].count("textabsatz") == 0:
                continue
            else:
                contentOfArticle = contentOfArticle + paragraph.text.strip()
                
        article.content = contentOfArticle
        
    print("--DONE--")    
    return
    




In [8]:
articles = getArt()
printArticles(articles)

Getting all Links...
Got 1 articles...
Getting content of article: Tabubruch Bundestag, Fehlgeburten, TikTok-Blockbuster
--DONE--
------
https://www.tagesschau.de/multimedia/podcast/15-minuten/audio-tabubruch-im-bundestag--schutz-nach-fehlgeburten-100.html
15 Minuten
Tabubruch Bundestag, Fehlgeburten, TikTok-Blockbuster

------


In [10]:
def saveToCsv(articles):
    dateNow = date.today()
    fileName = "TagesschauCrawl" + str(dateNow) + ".csv"
    with open(fileName, 'w', newline='', encoding='utf-8') as csvfile:
        articlewriter = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        for article in articles:
            articlewriter.writerow([article.topline, article.headline, article.content, article.link])

In [5]:
def saveToTxt(articles, filename):
    text = ""
    for article in articles:
        text += str(article.content)
    
    with open((filename), 'w') as file:
        file.write(text)
        
turnTextToOneText(articles, "WholeText")
    
saveToTxt(articles, "TagesschauToTxt.txt")

NameError: name 'turnTextToOneText' is not defined

In [ ]:
def mostCommonWordsFromTxt(filename, howManyMostCommonWords):
    with open(filename, 'r') as file:
        text = file.read()
    words = text.split()
    wordCounts = Counter(words)
    #print(wordCounts)
    mostCommonWords = wordCounts.most_common(howManyMostCommonWords)
    return mostCommonWords[:howManyMostCommonWords]
    
mostCommon = mostCommonWordsFromTxt("TagesschauToTxt.txt", 100)
for tuple in mostCommon:
    if tuple[0].islower() or "-" in tuple[0] or "+" in tuple[0]:
        continue
    else: print(tuple)
#print(mostCommon)

In [ ]:
def printMostCommonInHeadline(articles, howManyMostCommonWords):   
    headlineText = ""
    for article in articles:
        headLine = article.headline.split()
        for word in headLine:
            if word.islower() or "+" in word or "-" in word:
                continue
            else:
                if "?" in word:
                    word = word.replace("?", "")
                headlineText += word + " "

    words = headlineText.split()
    wordCounts = Counter(words)
    mostCommonHeadline = wordCounts.most_common(howManyMostCommonWords)
    print(mostCommonHeadline[:howManyMostCommonWords])
    
printMostCommonInHeadline(articles, 100)
